In [1]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd 
import numpy as np
from src.config import config, MODEL_CONFIG, PREP_CONFIG
from transformers import AutoTokenizer

In [3]:
# 1. Taking a look at cleaned dataset

In [4]:
stripped = pd.read_parquet(config.data.news.stripped)

In [5]:
stripped.columns

Index(['time', 'stocks', 'title', 'body', 'intra_day_time', 'channels',
       'company_name', 'short_name', 'author', 'parsed_body', 'cls_token',
       'staleness'],
      dtype='object')

In [6]:
stripped[["title", "stocks", "parsed_body"]].tail()

,title,stocks,parsed_body
index,,,
4578113,Cybin Announces Renewed At-The-Market Equity P...,CYBN,the company announces renewed atthemarket eq...
4578114,Environmental Icon Dr. Jane Goodall's Roots & ...,TM,environmental icon dr jane goodalls roots sho...
4578116,Chindata Group Unveils Pioneering Full-Stack S...,CD,chindata group unveils pioneering fullstack so...
4578117,Orbital Infrastructure Group Inc. Intends to E...,OIG,the company inc intends to enter into purcha...
4578118,"IBRX FINAL DEADLINE TUESDAY: ROSEN, TRUSTED IN...",IBRX,ibrx final deadline tuesday rosen trusted inve...


In [ ]:
# 2. Take a look at input_ids and if tokenizer works correctly

In [12]:
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)

In [13]:
ld = pd.read_parquet(config.data.learning_dataset)

In [14]:
#cleaned = ld = pd.read_parquet(config.data.merged) # requires more memory

In [3]:
tokenizer = AutoTokenizer.from_pretrained(PREP_CONFIG.tokenizer)

In [44]:
mask = ld.parsed_body.apply(lambda x: "today announced" in x)

In [47]:
ld = ld[mask]

In [ ]:
n = np.random.randint(0, ld.shape[0])
index = ld.index[n]
print(index)
entry = title_inputs_ids.loc[index, :].values

print(ld.loc[index, "stocks"])
print(f"{tokenizer.decode(entry)} \n")
print(ld.loc[index, "parsed_body"])

for x in entry:
    print(f"{x} --- {tokenizer.decode(x)}")


# Average number of token in news

In [4]:
title_inputs_ids = pd.read_parquet(config.data.news.input_ids)


In [5]:
n_padding = title_inputs_ids[title_inputs_ids == 0].sum(axis=1)

# Predictions from Model

In [4]:
ld = pd.read_parquet(config.data.learning_dataset, columns=["z_score", "z_score_class", "parsed_body"])
title_inputs_ids = pd.read_parquet(config.data.news.title_only.input_ids)
masks = pd.read_parquet(config.data.news.title_only.masks)

In [5]:
ld_mini = ld.iloc[-10000:,:]

In [6]:
index = ld.index[0]

In [7]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm


In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, BertTokenizer
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained("/gxfs_work/cau/sunms534/trading_bot/data/models/ProsusAI/finbert", num_labels=3, local_files_only=True)

prob = []
preds = []
actuals = []
with torch.no_grad():
    for index in tqdm(ld_mini.index):
        input_id = torch.tensor([title_inputs_ids.loc[index, :].tolist()], dtype=torch.int32)
        mask = torch.tensor([masks.loc[index, :].tolist()])
        result = finbert(input_id, mask)
        probs = F.softmax(result.logits).numpy()
        prob.append(probs.max())
        preds.append((probs.argmax()  + 1) % 2)
        actuals.append(ld.loc[index, "z_score_class"])
        #print(result)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

In [ ]:
F.softmax(result.logits).numpy().argmax()

In [75]:
tokenizer.decode(title_inputs_ids.loc[ld_mini.index[532]])

'[CLS] robotexts increase while robocalls drop for third consecutive month, according to robokiller insights [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
len(preds)

10000

In [58]:
df = pd.DataFrame(zip(preds,actuals, prob))

In [59]:
df.loc[:, 2].describe()

count    10000.000000
mean         0.825836
std          0.124616
min          0.384929
25%          0.757534
50%          0.875642
75%          0.923297
max          0.969893
Name: 2, dtype: float64

In [60]:
df = df[df.iloc[:, 2] > 0.95]

In [61]:
len(df)

227

In [62]:
(df.iloc[:, 0] == df.iloc[:, 1]).sum()

115

In [63]:
df.iloc[:, 1].value_counts()

1
1    119
2     65
0     43
Name: count, dtype: int64

In [64]:
df.iloc[:, 0].value_counts()

0
1    210
0     17
Name: count, dtype: int64

In [70]:
df.iloc[:, 2].idxmax()

532

In [72]:
df.loc[532, :]

0    0.000000
1    1.000000
2    0.969893
Name: 532, dtype: float64

In [74]:
ld.loc[ld_mini.index[532], "parsed_body"]

'Robotexts Increase While Robocalls Drop For Third Consecutive Month, According to Robokiller Insights. Americans received 14.2 billion robotexts and 4.6 billion robocalls in May, a 9% increase and 10% decrease, respectively, according to Robokiller Insights. The two categories continue to follow diverging paths — this marks the third consecutive month over month increase in SMS spam and the second consecutive month over month decrease in unwanted calls. Government anti robocall efforts continue to pay dividends The decrease in robocalls can be attributed largely to the government\'s successful efforts to crack down on specific high volume robocalls. Yet, although the quantity of robocalls is dropping, the damage they cause is escalating. Americans have lost $29 billion to robocalls already in 2023, outpacing 2022\'s record losses. Scammers are getting better results despite fewer calls, signifying they continue to refine their tactics and double down on scams that work. According to t

## Analyze impact of correction for standard deviation 

In [ ]:
import pandas as pd
from src.config import config, MODEL_CONFIG
import numpy as np

In [ ]:
dat: pd.DataFrame = pd.read_parquet(path=config.data.merged)

In [5]:
dat.columns

Index(['est_entry_time', 'est_exit_time', 'entry_time', 'exit_time', 'r',
       'r_spy', 'unadj_entry_open', 'entry_is_too_far_apart',
       'exit_is_too_far_apart', 'parsed_body', 'staleness', 'stocks',
       'std_252', 'dollar_volume', 'r_intra_(t-1)', 'unadj_open', 'cond_vola',
       'r_mkt_adj', 'z_score', 'z_score_class', 'sample_weights'],
      dtype='object')

In [9]:
X = dat[['r', 'r_spy', 'std_252', 'stocks', 'entry_time']]

In [12]:
X.loc[:, 'mkt_adj'] = X['r'] - X['r_spy']

In [15]:
X = X.dropna()

In [17]:
X.describe()

,r,r_spy,std_252,mkt_adj
count,2.337559e+06,2.337559e+06,2.337559e+06,2.337559e+06
mean,2.261430e-04,1.758870e-04,2.962545e-02,5.025593e-05
std,4.079177e-02,8.814592e-03,8.741242e-01,3.954428e-02
min,-8.825647e-01,-1.190273e-01,1.822642e-04,-8.848767e-01
25%,-1.253482e-02,-3.663569e-03,1.592303e-02,-1.129293e-02
50%,0.000000e+00,5.393307e-04,2.241463e-02,-1.961995e-04
75%,1.235995e-02,4.528339e-03,3.398496e-02,1.077793e-02
max,6.535211e+00,1.044661e-01,4.723933e+02,6.522799e+00


In [18]:
X.loc[:, 'z_score'] = X['mkt_adj'] / X['std_252']

/scratch/SlurmTMP/sunms534.10881239/ipykernel_2837370/2975249423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'z_score'] = X['mkt_adj'] / X['std_252']


In [20]:
X.sort_values('z_score', inplace=True)

/scratch/SlurmTMP/sunms534.10881239/ipykernel_2837370/373010617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values('z_score', inplace=True)


In [ ]:
X

In [28]:
X['z_score'].describe()

count    2.337559e+06
mean     1.480262e-04
std      1.107952e+00
min     -1.043610e+02
25%     -4.966595e-01
50%     -9.333168e-03
75%      4.859626e-01
max      1.195337e+02
Name: z_score, dtype: float64

In [29]:
X.loc[:, "z_score"] = X["z_score"].clip(lower=X["z_score"].quantile(0.05), upper=X["z_score"].quantile(0.95))

In [30]:
X['z_score'].describe()

count    2.337559e+06
mean    -2.617431e-03
std      7.952768e-01
min     -1.577498e+00
25%     -4.966595e-01
50%     -9.333168e-03
75%      4.859626e-01
max      1.602021e+00
Name: z_score, dtype: float64

Maximum ist 1.6, minimum ist -1.6 nach winsorisieren...

Problem, wenn wir mit std_252 standardisieren ist, dass es Wertpapiere (Aktien?) gibt mit sehr geringer Vola. Und weil wir die std von r nehmen, und nicht von r - r_spy bekommen wir unsinnige, viel zu große Werte heraus. -> Filtere minimale std_252 heraus.

In [31]:
X['std_252'].describe()

count    2.337559e+06
mean     2.962545e-02
std      8.741242e-01
min      1.822642e-04
25%      1.592303e-02
50%      2.241463e-02
75%      3.398496e-02
max      4.723933e+02
Name: std_252, dtype: float64

In [ ]:
X.sort_values('std_252')

In [ ]:
learndat: pd.DataFrame = pd.read_parquet(path=config.data.learning_dataset)

In [ ]:
learndat.sort_values("z_score")

In [ ]:
learndat.z_score.describe()

In [38]:
(learndat.z_score/learndat.std_252).describe()

count    954478.000000
mean         -0.003496
std           0.949972
min        -104.360953
25%          -0.497133
50%          -0.002012
75%           0.500479
max         119.533666
dtype: float64

## Staleness analysis

In [ ]:
dat: pd.DataFrame = pd.read_parquet(path=config.data.merged)

def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

dataset = dat

from tqdm.notebook import tqdm

# To determine the freshness of news, I compare the similarity of each news article with all articles published in the previous three days.
original_index_name = dataset.index.name
for ticker in tqdm(list(set(dataset.stocks))[1:100], desc="stocks"):
    orig_sort_ticker_news = dataset[dataset.stocks == ticker]

    # Using time sorted df makes for easier splicing later
    ticker_news = orig_sort_ticker_news.reset_index().set_index("est_entry_time").sort_index(ascending=True)
    
    # Set staleness of first news message to 0 
    ticker_news.at[ticker_news.index[0], "jaccard"] = 0

    for idx in orig_sort_ticker_news.index:
        time = orig_sort_ticker_news.at[idx, 'est_entry_time']
        previous_news = ticker_news.loc[(time-pd.DateOffset(days=3)):time-pd.DateOffset(minutes=1), "parsed_body"]
        if len(previous_news) == 0:
            ticker_news.at[time, "jaccard"] = 0
        else:
            current_str = orig_sort_ticker_news.at[idx, 'parsed_body']
            previous_news = previous_news
            jaccards = previous_news.apply(lambda x: jaccard_similarity(current_str, x))
            ticker_news.loc[ticker_news[original_index_name] == idx, "jaccard"] = jaccards.max()
            
    ticker_news.set_index(original_index_name, inplace=True)
    # Add entries to data set
    dataset.loc[ticker_news.index, "jaccard2"] = ticker_news.loc[:, "jaccard"]

In [59]:
tmp = dataset.dropna()

In [33]:
dataset[["jaccard", "staleness", 'jaccard2']].describe()

,jaccard,staleness
count,2.508859e+06,2.508859e+06
mean,2.551004e-01,6.638613e-01
std,3.450391e-01,4.696810e-01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.081081e-01,9.997681e-01
75%,2.760181e-01,1.000000e+00
max,1.000000e+00,1.000000e+00


In [34]:
import plotly.express as px

In [ ]:
px.histogram(dataset["jaccard"])

In [ ]:
px.histogram(dataset["staleness"])

In [ ]:
dataset[dataset.stocks == "ACCO"].loc[39023:]

In [66]:
jacc_df = tmp[tmp["jaccard2"] == 1]

In [ ]:
jacc_df[jacc_df.stocks == "ACCO"]